<a href="https://colab.research.google.com/github/reachaki/compintelgroup16/blob/main/compintel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate peft
import torch
import numpy as np
import random
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType


In [ ]:
# For reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)


In [ ]:
dataset = load_dataset('emotion')
print(dataset)


In [ ]:
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6).to('cpu')


In [ ]:
def tokenize_function(example):
    return tokenizer(
        example['text'],
        truncation=True,
        padding='max_length',
        max_length=128
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [ ]:
small_train = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_val = tokenized_datasets["validation"]  # or .select(range(500)) for even faster evaluation


In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    return {'eval_accuracy': acc}


In [ ]:
def train_lora_model(lora_rank, lora_alpha, lora_dropout, batch_size=16, learning_rate=2e-5, num_epochs=3):
    from transformers import AutoModelForSequenceClassification
    model = AutoModelForSequenceClassification.from_pretrained(
        'distilbert-base-uncased', num_labels=6).to('cpu')
    lora_config = LoraConfig(
        r=lora_rank,
        lora_alpha=lora_alpha,
        target_modules=["q_lin", "v_lin"],
        lora_dropout=lora_dropout,
        bias="none",
        task_type=TaskType.SEQ_CLS,
        inference_mode=False
    )
    lora_model = get_peft_model(model, lora_config)
    lora_model = lora_model.to('cpu')
    training_args = TrainingArguments(
        output_dir='./results',
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        logging_dir='./logs',
        report_to=[]
    )
    trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics
)

    trainer.train()
    eval_results = trainer.evaluate()
    accuracy = eval_results.get("eval_accuracy", None)
    print(f"Validation accuracy: {accuracy}")
    return accuracy


In [ ]:
train_lora_model(lora_rank=2, lora_alpha=8, lora_dropout=0.15, num_epochs=3)


In [ ]:
import random
import pandas as pd

# Define your search space
rank_choices = [2, 4, 8, 16]
alpha_choices = [8, 16, 32, 64]
dropout_choices = [0.05, 0.1, 0.15, 0.2]

results = []

num_trials = 5  # increase later for real experiments
for trial in range(num_trials):
    lora_rank = random.choice(rank_choices)
    lora_alpha = random.choice(alpha_choices)
    lora_dropout = random.choice(dropout_choices)
    print(f"Trial {trial+1}")
    val_acc = train_lora_model(lora_rank, lora_alpha, lora_dropout, num_epochs=1)
    results.append({
        'trial': trial+1,
        'lora_rank': lora_rank,
        'lora_alpha': lora_alpha,
        'lora_dropout': lora_dropout,
        'val_accuracy': val_acc
    })

# Save results to a DataFrame for easier analysis
results_df = pd.DataFrame(results)
print(results_df)
